In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [2]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [3]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5
# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
gridworld.show()

trajectorySteps = 100

value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
print(policy) #随机得到25*5的矩阵，每一行是一个state选择5个action的概率

gridworld.showPolicy(policy)
# gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
⬅️➡️⬇️⬆️➡️
⬆️⏬⏩️⬅️⬇️
⬅️🔄🔄⬆️⬇️
⬅️⏩️✅⏬⬇️
⬇️⏫️⬆️➡️🔄


In [4]:
# 通过采样的方法计算action value，model free的话意味着不知道整个gridworld,并不知道有多少个state，因此我们不再用上面的value
# 显示当前策略在网格世界中的情况
gridworld.showPolicy(policy)
# 打印提示信息，表示当前使用的是随机策略
print("random policy")
# 初始化一个全零的Q表，行数为网格世界的状态总数（rows * columns），列数为动作数（5）
qtable = np.zeros((rows*columns,5))
# 复制Q表，并给每个元素加1，作为上一轮的Q表，用于后续比较两次迭代的差异
qtable_pre = qtable.copy()+1
# 当两轮Q表的差异平方和大于0.001时，继续迭代更新Q表
while(np.sum((qtable_pre-qtable)**2)>0.001):
    # 打印两轮Q表的差异平方和，该值越小表示Q表越稳定
    print(np.sum((qtable_pre-qtable)**2)) 
    # 将当前Q表复制给上一轮的Q表，为下一轮迭代做准备
    qtable_pre = qtable.copy()
    # 通过采样获得action-value的值
    # 遍历所有状态
    for i in range(rows*columns):
        # 遍历所有动作
        for j in range(5):
            # 从当前状态i出发，执行动作j，根据策略policy采样一条轨迹，并返回轨迹的得分信息
            Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy, steps=trajectorySteps)
            
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            # a = r + gamma*r1 + gamma*gamma*r2 + gamma*gamma*gamma*r3 ……
            # 返回值是 S A R
            
            # 初始化临时变量，用于存储动作价值的累计回报，初始值为轨迹最后一步的奖励
            tmp = Trajectory[trajectorySteps][2]
            # 从倒数第二步开始，从后往前遍历轨迹，计算折扣累计回报
            for k in range(trajectorySteps-1,-1,-1):
                # 根据折扣因子gamma更新临时变量，累加每一步的奖励
                tmp = tmp*gamma + Trajectory[k][2]  # 细节从后往前优化算法
            # print(tmp)
            
            # 将计算得到的动作价值更新到Q表中对应的位置，Q表的索引是状态i和动作j，相当于从任何一个状态i出发，执行动作j，得到的动作价值
            qtable[i][j] = tmp # 这里是通过采样，获得action value的值

    # 根据更新后的Q表，选择每个状态下的最优动作，并将策略更新为对应的独热编码,policy的shape是(rows*columns,5)，单步可以看到如何把sparse id变为one-hot
    policy = np.eye(5)[np.argmax(qtable,axis=1)]
    # 打印状态17对应的Q表值
    print(qtable[17])
    # 打印状态22对应的Q表值
    print(qtable[22])
    # policy = np.eye(5)[np.argmax(qtable,axis=1)]  # qtable的最优值作为更新策略，并用独热码来表示
    # 显示更新策略后在网格世界中的情况
    gridworld.showPolicy(policy)
    # 打印更新后两轮Q表的差异平方和
    print(f'Q差异{np.sum((qtable_pre-qtable)**2)}')


⬅️➡️⬇️⬆️➡️
⬆️⏬⏩️⬅️⬇️
⬅️🔄🔄⬆️⬇️
⬅️⏩️✅⏬⬇️
⬇️⏫️⬆️➡️🔄
random policy
125.0


[-99.99760947 -10.           4.73671629  -5.26328371   5.26304466]
[  5.26304466   0.           3.73671629 -14.73695534   4.73671629]
⬇️⬅️➡️➡️⬇️
🔄⏬⏪➡️⬇️
➡️🔄⏬➡️⬆️
➡️⏩️✅⏪⬆️
⬆️⏩️⬆️⬅️⬆️
Q差异109873.89400845372
109873.89400845372
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
🔄⏩️⬆️⬅️⬅️
Q差异99729.59305303964
99729.59305303964
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
Q差异678.0376617226709
678.0376617226709
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
Q差异215.2179210453017
215.2179210453017
[-1.00023905 -1.00023905  8.99976095 -1.00023905  9.99976095]
[ 9.99976095  8.09976095  7.99976095 -1.90023905  8.99976095]
➡️➡️➡️➡️⬇

In [5]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])